# Spatial Regression with PySAL 2.0

Vitor Kamada

Last change: 9/28/2019

The baseline code here is from Rey and Arribas-Bel (2018). Geographic Data
Science with PySAL.
http://darribas.org/gds_scipy16/

Rey and Arribas-Bel (2018) wrote the code  to run in Anaconda, using PySAL 1.11. 

I added and modified many parts of the code to run in Google Colab and PySAL 2.0.


# Installing spatialindex, rtree, and pysal 2.0.

In [0]:
!apt-get install -qq curl g++ make

In [2]:
!curl -L http://download.osgeo.org/libspatialindex/spatialindex-src-1.8.5.tar.gz | tar xz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  556k  100  556k    0     0   675k      0 --:--:-- --:--:-- --:--:--  674k


In [0]:
import os
os.chdir('spatialindex-src-1.8.5')

In [4]:
!./configure

checking build system type... x86_64-unknown-linux-gnu
checking for g++... g++
checking whether the C++ compiler works... yes
checking for C++ compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether we are using the GNU C++ compiler... yes
checking whether g++ accepts -g... yes
checking how to run the C++ preprocessor... g++ -E
checking for a BSD-compatible install... /usr/bin/install -c
checking whether ln -s works... yes
checking whether make sets $(MAKE)... yes
checking host system type... x86_64-unknown-linux-gnu
checking how to print strings... printf
checking for gcc... gcc
checking whether we are using the GNU C compiler... yes
checking whether gcc accepts -g... yes
checking for gcc option to accept ISO C89... none needed
checking whether gcc understands -c and -o together... yes
checking for a sed that does not truncate output... /bin/sed
checking fo

In [5]:
!make

Making all in src
make[1]: Entering directory '/content/spatialindex-src-1.8.5/src'
Making all in storagemanager
make[2]: Entering directory '/content/spatialindex-src-1.8.5/src/storagemanager'
make[2]: Nothing to be done for 'all'.
make[2]: Leaving directory '/content/spatialindex-src-1.8.5/src/storagemanager'
Making all in spatialindex
make[2]: Entering directory '/content/spatialindex-src-1.8.5/src/spatialindex'
make[2]: Nothing to be done for 'all'.
make[2]: Leaving directory '/content/spatialindex-src-1.8.5/src/spatialindex'
Making all in rtree
make[2]: Entering directory '/content/spatialindex-src-1.8.5/src/rtree'
make[2]: Nothing to be done for 'all'.
make[2]: Leaving directory '/content/spatialindex-src-1.8.5/src/rtree'
Making all in mvrtree
make[2]: Entering directory '/content/spatialindex-src-1.8.5/src/mvrtree'
make[2]: Nothing to be done for 'all'.
make[2]: Leaving directory '/content/spatialindex-src-1.8.5/src/mvrtree'
Making all in tprtree
make[2]: Entering directory '/co

In [6]:
!make install

Making install in src
make[1]: Entering directory '/content/spatialindex-src-1.8.5/src'
Making install in storagemanager
make[2]: Entering directory '/content/spatialindex-src-1.8.5/src/storagemanager'
make[3]: Entering directory '/content/spatialindex-src-1.8.5/src/storagemanager'
make[3]: Nothing to be done for 'install-exec-am'.
make[3]: Nothing to be done for 'install-data-am'.
make[3]: Leaving directory '/content/spatialindex-src-1.8.5/src/storagemanager'
make[2]: Leaving directory '/content/spatialindex-src-1.8.5/src/storagemanager'
Making install in spatialindex
make[2]: Entering directory '/content/spatialindex-src-1.8.5/src/spatialindex'
make[3]: Entering directory '/content/spatialindex-src-1.8.5/src/spatialindex'
make[3]: Nothing to be done for 'install-exec-am'.
make[3]: Nothing to be done for 'install-data-am'.
make[3]: Leaving directory '/content/spatialindex-src-1.8.5/src/spatialindex'
make[2]: Leaving directory '/content/spatialindex-src-1.8.5/src/spatialindex'
Making i

In [7]:
!pip install rtree

In [0]:
!ldconfig

Working through the example at http://toblerity.org/rtree/examples.html

In [0]:
from rtree import index
from rtree.index import Rtree

In [10]:
p = index.Property()
idx = index.Index(properties=p)
idx

In [11]:
pip install -U pysal

Requirement already up-to-date: pysal in /usr/local/lib/python3.6/dist-packages (2.1.0)


# Loading PySAL 2.0

In [12]:
from pysal.model import spreg
from pysal.lib import weights
from pysal.explore import esda
from scipy import stats
import statsmodels.formula.api as sm
import numpy as np
import pandas as pd
import geopandas
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/pysal/explore/segregation/network/network.py:16: UserWarning: You need pandana and urbanaccess to work with segregation's network module
You can install them with  `pip install urbanaccess pandana` or `conda install -c udst pandana urbanaccess`
  "You need pandana and urbanaccess to work with segregation's network module\n"


# Austin Properties Listed in AirBnb

http://insideairbnb.com/austin/index.html

In [0]:
abb_link = 'https://github.com/VitorKamada/ECO6100/raw/master/Data/Texas/listings.csv.gz'
lst = pd.read_csv(abb_link)

In [14]:
lst.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,...,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,72635,https://www.airbnb.com/rooms/72635,20151107173015,2015-11-08,"3 Private Bedrooms, SW Austin",Conveniently located 10-15 from downtown in SW...,"We have three spare bedrooms, each with a quee...",Conveniently located 10-15 from downtown in SW...,none,Location and convenience are key. Easy access...,NaN,Unfortunately there is no convenient public tr...,https://a2.muscache.com/ac/pictures/69586491/a...,https://a2.muscache.com/im/pictures/69586491/a...,https://a2.muscache.com/ac/pictures/69586491/a...,https://a2.muscache.com/ac/pictures/69586491/a...,377044,https://www.airbnb.com/users/show/377044,Jason & Marcie,2011-02-04,"Austin, Texas, United States",Private music loving family in SW Austin,NaN,NaN,NaN,f,https://a2.muscache.com/ac/users/377044/profil...,https://a2.muscache.com/ac/users/377044/profil...,Circle C,1.0,1.0,"['email', 'facebook', 'linkedin', 'reviews']",t,f,"West Slaughter Lane, Austin, TX 78749, United ...",Circle C,78739,NaN,Austin,TX,...,1.0,3.0,Real Bed,"{""Cable TV"",Internet,""Wireless Internet"",""Air ...",3600.0,$300.00,NaN,NaN,NaN,NaN,1,$0.00,2,10,8 months ago,t,0,0,0,240,2015-11-07,1,2011-03-21,2011-03-21,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f,NaN,NaN,f,moderate,f,f,1,0.02
1,5386323,https://www.airbnb.com/rooms/5386323,20151107173015,2015-11-07,Cricket Trailer,Rent this cool concept trailer that has everyt...,Rental arrangements for this trailer allows yo...,Rent this cool concept trailer that has everyt...,none,We're talking about wherever you'd like in the...,NaN,"Bike, Bus, Metrorail, etc. you name it we've g...",https://a0.muscache.com/ac/pictures/67298119/7...,https://a0.muscache.com/im/pictures/67298119/7...,https://a0.muscache.com/ac/pictures/67298119/7...,https://a0.muscache.com/ac/pictures/67298119/7...,27922117,https://www.airbnb.com/users/show/27922117,Michael,2015-02-17,"Austin, Texas, United States",Family man that loves the great outdoors.,NaN,NaN,NaN,f,https://a1.muscache.com/ac/users/27922117/prof...,https://a1.muscache.com/ac/users/27922117/prof...,Circle C,1.0,1.0,"['email', 'phone', 'google', 'kba']",t,t,"Seneca Falls Loop, Austin, TX 78739, United St...",Circle C,78739,NaN,Austin,TX,...,1.0,1.0,Futon,"{""Air Conditioning"",Heating,""Family/Kid Friend...",NaN,$99.00,$600.00,NaN,"$1,000.00",$75.00,1,$0.00,3,7,8 months ago,t,29,59,89,364,2015-11-07,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN
2,8826517,https://www.airbnb.com/rooms/8826517,20151107173015,2015-11-07,Private room 1 in South Austin,"Upstairs, private, 12ft x 13 1/2ft room. Priv...",NaN,"Upstairs, private, 12ft x 13 1/2ft room. Priv...",none,NaN,NaN,NaN,https://a0.muscache.com/ac/pictures/111684503/...,https://a0.muscache.com/im/pictures/111684503/...,https://a0.mu

In [0]:
x = ['host_listings_count', 'bathrooms', 
     'bedrooms', 'beds', 'guests_included']

In [0]:
def has_pool(a):
    if 'Pool' in a:
        return 1
    else:
        return 0

In [0]:
lst['pool'] = lst['amenities'].apply(has_pool)


In [0]:
yxs = lst.loc[:, x + ['pool', 'price']].dropna()
y = np.log(yxs['price'].apply(lambda x: 
      float(x.strip('$').replace(',', '')))
      + 0.000001)

In [19]:
w = weights.KNN.from_array(lst.loc[yxs.index, 
       ['longitude', 'latitude']].values)
w.transform = 'R'

/usr/local/lib/python3.6/dist-packages/pysal/lib/weights/weights.py:170: UserWarning: The weights matrix is not fully connected. There are 360 components
  warnings.warn("The weights matrix is not fully connected. There are %d components" % self.n_components)


In [20]:
w

In [21]:
m1 = spreg.OLS(y.values[:, None], 
        yxs.drop('price', axis=1).values, 
        w=w, spat_diag=True, 
                name_x=yxs.drop('price',
                axis=1).columns.tolist(), 
                 name_y='ln(price)')
print(m1.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :   ln(price)                Number of Observations:        5767
Mean dependent var  :      5.1952                Number of Variables   :           7
S.D. dependent var  :      0.9455                Degrees of Freedom    :        5760
R-squared           :      0.4042
Adjusted R-squared  :      0.4036
Sum squared residual:    3071.189                F-statistic           :    651.3958
Sigma-square        :       0.533                Prob(F-statistic)     :           0
S.E. of regression  :       0.730                Log likelihood        :   -6366.162
Sigma-square ML     :       0.533                Akaike info criterion :   12746.325
S.E of regression ML:      0.7298                Schwarz criterion     :   12792.944

-----------------------------------------------------------------------------

In [22]:
w_pool = weights.KNN.from_array(lst.loc[yxs.index, 
                    ['longitude', 'latitude']].values)

/usr/local/lib/python3.6/dist-packages/pysal/lib/weights/weights.py:170: UserWarning: The weights matrix is not fully connected. There are 360 components
  warnings.warn("The weights matrix is not fully connected. There are %d components" % self.n_components)


In [0]:
yxs_w = yxs.assign(w_pool=weights.lag_spatial(w_pool,
                                  yxs['pool'].values))


In [24]:
m2 = spreg.OLS(y.values[:, None], 
        yxs_w.drop('price', axis=1).values, 
  w=w, spat_diag=True, 
  name_x=yxs_w.drop('price', axis=1).columns.tolist(), 
  name_y='ln(price)')

print(m2.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :   ln(price)                Number of Observations:        5767
Mean dependent var  :      5.1952                Number of Variables   :           8
S.D. dependent var  :      0.9455                Degrees of Freedom    :        5759
R-squared           :      0.4044
Adjusted R-squared  :      0.4037
Sum squared residual:    3070.363                F-statistic           :    558.6139
Sigma-square        :       0.533                Prob(F-statistic)     :           0
S.E. of regression  :       0.730                Log likelihood        :   -6365.387
Sigma-square ML     :       0.532                Akaike info criterion :   12746.773
S.E of regression ML:      0.7297                Schwarz criterion     :   12800.053

-----------------------------------------------------------------------------

In [25]:
m3 = spreg.GM_Lag(y.values[:, None], 
   yxs.drop('price', axis=1).values, 
   w=w, spat_diag=True, 
   name_x=yxs.drop('price', 
            axis=1).columns.tolist(), 
   name_y='ln(price)')

print(m3.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :   ln(price)                Number of Observations:        5767
Mean dependent var  :      5.1952                Number of Variables   :           8
S.D. dependent var  :      0.9455                Degrees of Freedom    :        5759
Pseudo R-squared    :      0.4224
Spatial Pseudo R-squared:  0.4056

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       3.7085715       0.1075621      34.4784213       0.0000000
 host_listings_count      -0.0000587       0.0001765      -0.3324585       0.7395430
           bathrooms       0.2857932       0.0193237      1

In [26]:
b = m3.betas[:-1]
b

array([[ 3.70857151e+00],
       [-5.86948398e-05],
       [ 2.85793233e-01],
       [ 3.27259811e-01],
       [ 2.39548235e-02],
       [ 6.51467596e-03],
       [ 8.91100274e-02]])

In [27]:
rho = m3.betas[-1]
rho

array([0.07850592])

In [0]:
btot = b / (1.0 - rho) #total impact
bind = btot - b #indirect impact

In [29]:
x_names = ['NROOM','NBATH','PATIO','FIREPL','AC','GAR','AGE',
           'LOTSZ','SQFT']
varnames = ["CONSTANT"] + x + ["pool"]
print("            Variable       Direct       Indirect  \
    Total" )
for i in range(len(varnames)):
    print("%20s %12.3f %12.3f %12.3f" % (varnames[i],b[i][0],
                                bind[i][0],btot[i][0]))


            Variable       Direct       Indirect      Total
            CONSTANT        3.709        0.316        4.025
 host_listings_count       -0.000       -0.000       -0.000
           bathrooms        0.286        0.024        0.310
            bedrooms        0.327        0.028        0.355
                beds        0.024        0.002        0.026
     guests_included        0.007        0.001        0.007
                pool        0.089        0.008        0.097
